In [1]:
import numpy as np
import sympy as sm
import matplotlib.pyplot as plt
import ipywidgets as widgets
from scipy import optimize

In this model project we want to solve the Solow model with humancapital. We take the model from 'Introducing Advanced Macroeconomics - Growth and Business Cycles'. 3 edition 2022, Sørensen, Peter Birch and Whitta-Jacobsen, Hans Jørgen.

The Solow model with human capital from Mankiw, Weil and Romer is an extenstion of the general Solow model, that helps explain why international investments does not flow to poorer countries as predicted by the general Solow model. The modification finds itself in the addition of human capital, which is mostly measured as years of education and experience in general, and sheds greater light on why some countries are rich and some are poor. 

# The model

In [2]:
#First we define all the parameters and variables using sympy
Y = sm.symbols('Y_t')           #Output
K = sm.symbols('K_t')           #Physical capital
alpha = sm.symbols('alpha')     #Income share of capital and labour
H = sm.symbols('H_t')           #Human Capital
phi = sm.symbols('phi')         #Income share of Human capital
A = sm.symbols('A_t')           #Technology
L = sm.symbols('L_t')           #Labour
r = sm.symbols('r_t')           #Real rentan rate on capital
w = sm.symbols('w_t')           #Real wage
n = sm.symbols('n')             #Growth rate of Labour
g = sm.symbols('g')             #Growth rate of technology
K1 = sm.symbols('K_{t+1}')      #Capital in the next period
sk = sm.symbols('s_K')          #Savings rate on physical capital
sh = sm.symbols('s_H')          #Savings rate on human capital
H1 = sm.symbols('H_{t+1}')      #Human capital in the next period
delta = sm.symbols('delta')     #Depreciation rate (both on physical and human capital!)
L1 = sm.symbols('L_{t+1}')      #Labour in the next period
A1 = sm.symbols('A_{t+1}')      #Technology in the next period
tau = sm.symbols('tau')         #Tax rate (to be used in the extended model)

In [3]:
#Here we create all the equations of the model and displays them again using sympy
Y_t = sm.Eq(Y, (K**alpha*H**phi)*(A*L)**(1-alpha-phi))
r_t = sm.Eq(r, alpha*(K/(A*L))**(alpha-1)*(H/(A*L))**phi)
w_t = sm.Eq(w, (1-alpha)*(K/(A*L))**alpha*(H/(A*L))**phi*A)
K_t1 = sm.Eq(K1, sk*Y+(1-delta)*K)
H_t1 = sm.Eq(H1, sh*Y+(1-delta)*H)
L_t1 = sm.Eq(L1, (1+n)*L)
A_t1 = sm.Eq(A1, (1+g)*A)
display(Y_t, r_t, w_t, K_t1, H_t1, L_t1, A_t1)

Eq(Y_t, H_t**phi*K_t**alpha*(A_t*L_t)**(-alpha - phi + 1))

Eq(r_t, alpha*(H_t/(A_t*L_t))**phi*(K_t/(A_t*L_t))**(alpha - 1))

Eq(w_t, A_t*(H_t/(A_t*L_t))**phi*(K_t/(A_t*L_t))**alpha*(1 - alpha))

Eq(K_{t+1}, K_t*(1 - delta) + Y_t*s_K)

Eq(H_{t+1}, H_t*(1 - delta) + Y_t*s_H)

Eq(L_{t+1}, L_t*(n + 1))

Eq(A_{t+1}, A_t*(g + 1))

The Solow model with human capital is given with these equations. 1) stating that output is given as a Cobb-Douglas production function. 2 and 3) stating that the real rental rate on capital and real wage is given from the marginalproduct of capital and labour respectively. 4 and 5) stating the accumulation equation of capital and human capital. 6) stating that labour grows with the exogeneously rate of n. 7) stating that technology grows with the exogeneously rate of g.

# Solving the model symbolically 

First we would like to solve the model symbolically with sympy meaning that we find the steady state values to thereafter solve for different values of steady state. We begin by deriving the transition equation and thereafter we define the Solow eqautions.

In [4]:
#First we define the numerous new symbols with sympy
h_tilde = sm.symbols('htilde_t')       #Human capital per effective worker
h_tilde1 = sm.symbols('htilde_{t+1}')      #Human capital per worker in the next period
k_tilde = sm.symbols('ktilde_t')       #Capital per effective worker
k_tilde1 = sm.symbols('ktilde_{t+1}')  #Capital per effective worker in next period

The law of motion in the Solow model with human capital can be descibed by the Solow equations. The Solow equations states how all of the parameters in the model move over time.

Because of sympys lack of operand types in its equation module we 'just' state the Solow equations for human- and capital below.

\begin{equation}
\tilde{k}_{t+1} - \tilde{k}_t = \frac{1}{(1+n)(1+g)}(s_K \tilde{k}_t^{\alpha} \tilde{h}_t^{\phi} - (n+g+\delta +ng)\tilde{k}_t)
\end{equation}
\begin{equation}
\tilde{h}_{t+1} - \tilde{h}_t = \frac{1}{(1+n)(1+g)}(s_H \tilde{k}_t^{\alpha} \tilde{h}_t^{\phi} - (n+g+\delta +ng)\tilde{h}_t)
\end{equation}

As we know analytically these equations can be solved to find steady state by setting $\tilde{k}_{t+1}$ and $\tilde{k}_{t}$ equal to eachother and solving for $\tilde{k}_{t}$ and the same for $\tilde{h}_{t+1}$ and $\tilde{h}_{t}$. That is the same as solving the right hand side and the left hand side in the big parenthesis to be equal to eachother.

In [5]:
#Here we create the equations to be solved
h_tilde_phi = h_tilde**phi
k_tilde_alpha = k_tilde**alpha
k = sm.Eq(sk * k_tilde_alpha * h_tilde_phi - (n + g + delta + n * g) * k_tilde, 0)
h = sm.Eq(sh * k_tilde_alpha * h_tilde_phi - (n + g+ delta + n * g) * h_tilde, 0)
display(k, h)

Eq(htilde_t**phi*ktilde_t**alpha*s_K - ktilde_t*(delta + g*n + g + n), 0)

Eq(-htilde_t*(delta + g*n + g + n) + htilde_t**phi*ktilde_t**alpha*s_H, 0)

Now we can solve this for $\tilde{k}_{t}$ and $\tilde{h}_{t}$ respectively by isolating $\tilde{k}_{t}$ in the first eqaution and $\tilde{h}_{t}$ in the second equation.

In [6]:
k_solve = sm.solve(k, k_tilde)[0]
h_solve = sm.solve(h, h_tilde)[0]
display(k_solve, h_solve)

((delta + g*n + g + n)/(htilde_t**phi*s_K))**(1/(alpha - 1))

((delta + g*n + g + n)/(ktilde_t**alpha*s_H))**(1/(phi - 1))

Now that we know what $\tilde{k}_{t}$ and $\tilde{h}_{t}$ is equal to we insert this in the k and h equations respectively.

In [7]:
#For k_tilde we inset the above equation onto k_tilde place in the k eqaution
k_solve_alpha = k_solve**alpha
k_inset = sm.Eq(sk * (k_solve**alpha) * (h_tilde**phi) - (n + g + delta + n * g) * k_solve, 0)
#For h_tilde we do the same. So inset it in the h equation
h_solve_phi = h_solve**phi
h_inset = sm.Eq(sh * h_solve_phi * k_tilde_alpha - (n + g+ delta + n * g) * h_solve, 0)
display(k_inset, h_inset)

Eq(htilde_t**phi*s_K*(((delta + g*n + g + n)/(htilde_t**phi*s_K))**(1/(alpha - 1)))**alpha - ((delta + g*n + g + n)/(htilde_t**phi*s_K))**(1/(alpha - 1))*(delta + g*n + g + n), 0)

Eq(ktilde_t**alpha*s_H*(((delta + g*n + g + n)/(ktilde_t**alpha*s_H))**(1/(phi - 1)))**phi - ((delta + g*n + g + n)/(ktilde_t**alpha*s_H))**(1/(phi - 1))*(delta + g*n + g + n), 0)

These are pretty convuluted equations and sympys is not able to solve for these equations. We will therefore have to simplify them by hand. We end up with these equations which is now solvable by sympy.

In [8]:
k_iso_sipml = sm.Eq((sk**2)*(delta+n*g+n+g)

SyntaxError: unexpected EOF while parsing (1147572729.py, line 1)

Now all that is left to do is to solve for $\tilde{h}_{t}$ and $\tilde{k}_{t}$ respectively and we get the steady state value for $\tilde{h}_{t}$ and $\tilde{k}_{t}$.

In [9]:
k_ss = sm.solve(k_inset, h_tilde)[0]
h_ss = sm.solve(h_inset, k_tilde)[0]
display(k_ss, h_ss)

NotImplementedError: multiple generators [((delta/(htilde_t**phi*s_K) + g*n/(htilde_t**phi*s_K) + g/(htilde_t**phi*s_K) + n/(htilde_t**phi*s_K))**(1/(alpha - 1)))**alpha, (delta/(htilde_t**phi*s_K) + g*n/(htilde_t**phi*s_K) + g/(htilde_t**phi*s_K) + n/(htilde_t**phi*s_K))**(1/(alpha - 1)), htilde_t**phi]
No algorithms are implemented to solve equation htilde_t**phi*s_K*(((delta + g*n + g + n)/(htilde_t**phi*s_K))**(1/(alpha - 1)))**alpha - ((delta + g*n + g + n)/(htilde_t**phi*s_K))**(1/(alpha - 1))*(delta + g*n + g + n) + 0

So we get our steady state values of $\tilde{k}_{t}$ and $\tilde{h}_{t}$ to be:

In [ ]:
#print('h_tilde^* is {hss})
#print('k_tilde^* is {kss})

We can now find $\tilde{y^*}$ out of our definition on $\tilde{y^{*}}$ = $\frac{y^{*}}{A_{{t}}}$  <=> ${y^*}$ = ${A}_{t}*\tilde{y^{*}}$

# Interactive phase diagram

# Solving the model numerically

# Interactive phase diagram

# Extended model with taxation/government 

We extend the model by including taxation on wealth and thereby adding an 'active' government. We add a taxation on physical capital (wealth) and add a government who uses all of the tax provenue to invest in human capital. Meaning that human capital will now only accumulate on the basis of physical capital, the government and the tax rate.

The extended Solow model with human capital and taxation is given by:

In [10]:
K_t_1 = sm.Eq(K1, sk+Y+(1-tau-delta)*K)
H_t_1 = sm.Eq(H1, tau*K+(1-tau)*H)
display(Y_t, L_t1, A_t1, K_t_1, H_t_1)

Eq(Y_t, H_t**phi*K_t**alpha*(A_t*L_t)**(-alpha - phi + 1))

Eq(L_{t+1}, L_t*(n + 1))

Eq(A_{t+1}, A_t*(g + 1))

Eq(K_{t+1}, K_t*(-delta - tau + 1) + Y_t + s_K)

Eq(H_{t+1}, H_t*(1 - tau) + K_t*tau)

The extended Solow model is given by these equations. 1) Stating that output is a generel Cobb-Douglas production function. 2 and 3) Stating that labour and technology grows with the exogenously rates of n and g respectively. 4) Stating that capital accumulates by the total amount of savings and last years capital, now including a tax rate. 5) Stating that human capital know accumulates only by the amount of tax provenue from physical capital and last periods amount of human capital.

# Solving the model symbolically

The solving of this model is a pendant to the baseline model and will therefore be less rich in text

The Solow equations for this model is now given by:

\begin{equation} 
\tilde{k}_{t+1} - \tilde{k}_t = \frac{1}{(1+n)(1+g)}(s_K \tilde{k}_t^{\alpha} \tilde{h}_t^{\phi} - (n+g+\tau+\delta +ng)\tilde{k}_t)
\end{equation}
\begin{equation}
\tilde{h}_{t+1} - \tilde{h}_t = \frac{1}{(1+n)(1+g)}(\tau \tilde{k}_t - (n+g+\delta +ng)\tilde{h}_t)
\end{equation}

As before the steady state can be calulated as the two sides of the negative sign in the big parenthesis should be equal to eachother. We make this creation with sympy.

In [11]:
k2 = sm.Eq(sk*(k_tilde**alpha)*(h_tilde**phi)-(n+g+tau+delta+n*g)*k_tilde, 0)
h2 = sm.Eq(tau*k_tilde-(n+g+delta+n*g)*h_tilde,0)
display(k2,h2)

Eq(htilde_t**phi*ktilde_t**alpha*s_K - ktilde_t*(delta + g*n + g + n + tau), 0)

Eq(-htilde_t*(delta + g*n + g + n) + ktilde_t*tau, 0)

Now we can isolate $\tilde{k}_{t}$ in the first eqaution and $\tilde{h}_{t}$ in the second equation

In [12]:
k_isolated = sm.solve(k2, k_tilde)[0]
h_isolated = sm.solve(h2, h_tilde)[0]
display(k_isolated, h_isolated)

((delta + g*n + g + n + tau)/(htilde_t**phi*s_K))**(1/(alpha - 1))

ktilde_t*tau/(delta + g*n + g + n)

Now we can insert this into the k2 and h2 equation like this:

In [13]:
h_isolated_phi = k_isolated**phi
k_inset2 = sm.Eq(sk * (k_tilde**alpha) * (h_isolated**phi) - (n + g + delta + tau + n * g) * k_tilde, 0)
#For h_tilde we do the same. So insert it in the h equation
h_inset2 = sm.Eq(tau * k_isolated - (n + g+ delta + n * g) * h_tilde, 0)
display(k_inset2, h_inset2)

Eq(-ktilde_t*(delta + g*n + g + n + tau) + ktilde_t**alpha*s_K*(ktilde_t*tau/(delta + g*n + g + n))**phi, 0)

Eq(-htilde_t*(delta + g*n + g + n) + tau*((delta + g*n + g + n + tau)/(htilde_t**phi*s_K))**(1/(alpha - 1)), 0)

These are pretty long and convulted equations. Sympy is not able to handle this because of lack of algorithms to solve for. Therefore we simplify this equations above analytically so sympy will be able to handle them. We do this by simplifying for $\tilde{k}_{t}$ in the first equation and for $\tilde{h}_{t}$ in the second equation.

In [14]:
k_inset3 = sm.Eq(((sk*((tau/(delta+n*g+n+g))**phi))/(k_tilde**(1-phi-alpha))), (n+g+delta+tau+n*g))
h_inset3 = sm.Eq((tau*(((delta+n*g+g+n+tau)/sk)**(1/(alpha-1))))/(h_tilde**((phi+alpha-1)/(alpha-1))),(delta+n+g+n*g))
display(k_inset3, h_inset3)

Eq(ktilde_t**(alpha + phi - 1)*s_K*(tau/(delta + g*n + g + n))**phi, delta + g*n + g + n + tau)

Eq(tau*((delta + g*n + g + n + tau)/s_K)**(1/(alpha - 1))/htilde_t**((alpha + phi - 1)/(alpha - 1)), delta + g*n + g + n)

Now all that is left to do is solve these two equations above with regards to $\tilde{k}_{t}$ and $\tilde{h}_{t}$ respectively to find the steady state values for $\tilde{k}_{t}$ and $\tilde{h}_{t}$

In [15]:
k_ss2 = sm.solve(k_inset3, k_tilde)[0]
h_ss2 = sm.solve(h_inset3, h_tilde)[0]
display(k_ss2, h_ss2)

((delta + g*n + g + n + tau)/(s_K*(tau/(delta + g*n + g + n))**phi))**(1/(alpha + phi - 1))

(tau*((delta + g*n + g + n + tau)/s_K)**(1/(alpha - 1))/(delta + g*n + g + n))**((alpha - 1)/(alpha + phi - 1))